### Домашнее задание

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

По шагам:
0. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
1. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
3. завести себе аккаунт на github (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
4. итоговый проект должен содержать: 1) каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна) 2) файл app/run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта) 4) README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д) 5) Dockerfile 6) docker-entrypoint.sh
5. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

Полезные ссылки:
1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой такой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data Там же и ноутбуки с разными подходами есть.
3. минималистичный пример связки keras/flask https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html для определения класса картинки
4. неплохой такой пример (помимо того, что разобрали на занятии) связки docker/flask - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04

p.s. если проблемы с выбором датасета, то пишите пожалуйста - будем вместе думать)

## Step 1 - TRAIN

### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/competitions/titanic
2. Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

%matplotlib inline
import matplotlib.pylab as plt

In [2]:
#Создание маленькой выборки для отладки кода

#df = pd.read_csv("train.csv")
#df1 = df.sample(n=90000, random_state=1)
#df1.to_csv("train_mini.csv", index=None)

In [3]:
# Загрузим данные

df = pd.read_csv("train.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
#Выясним количество классов

df['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [5]:
#Определим дубликаты

df.duplicated().sum()

#Дубликатов нет

0

In [6]:
#df['Age'] = df['Age'].fillna(df.Age.mode()[0])
#df['Cabin'] = df['Cabin'].fillna('Unknown')
#df['Embarked'] = df['Embarked'].fillna(df.Embarked.mode()[0])

In [7]:
#Разделим данные на train/test и сохраним тестовую выборку на диск

X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', 1), df['Survived'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)
y_train

6      0
718    0
685    0
73     0
882    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 596, dtype: int64

In [8]:
#Проанализируем данные

X_train.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [9]:
#Посмотрим выбросы по координатам

X_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,596.000000,596.000000,478.000000,596.000000,596.000000,596.000000
mean,448.508389,2.337248,29.525983,0.577181,0.374161,31.912786
std,259.457226,0.823207,14.457437,1.229504,0.807072,51.480961
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,221.750000,2.000000,20.250000,0.000000,0.000000,7.925000
50%,459.500000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,676.250000,3.000000,38.000000,1.000000,0.000000,31.275000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
#Проверим датасет на наличие пустых значений

X_train.info()

#Пустых значений нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596 entries, 6 to 102
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  596 non-null    int64  
 1   Pclass       596 non-null    int64  
 2   Name         596 non-null    object 
 3   Sex          596 non-null    object 
 4   Age          478 non-null    float64
 5   SibSp        596 non-null    int64  
 6   Parch        596 non-null    int64  
 7   Ticket       596 non-null    object 
 8   Fare         596 non-null    float64
 9   Cabin        134 non-null    object 
 10  Embarked     595 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 55.9+ KB


In [11]:
#Age заменяем (Pclass=3)+(Sex=male)
#Cabin заменяем (Pclass=1)+(Fare=mode)+(Embarked=B%)
#Embarked заменяем (Pclass=1)+(Fare=80)+(Cabin=B%)

In [12]:
# соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.key == 'Cabin':
            X[self.key] = X[self.key].fillna('Unknown')
        X[self.key] = X[self.key].fillna(X[self.key].mode()[0])
        return X[[self.key]]
    

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [13]:
#Определим признаки и цели
#Зададим списки признаков
#['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

categorical_columns = ['Sex', 'Cabin', 'Embarked']
continuous_columns = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [14]:
#Создадим трансформеры под каждый признак

from sklearn.preprocessing import StandardScaler

final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))
    
#for text_col in text_columns:
#    text_transformer = Pipeline([
#                ('imputer', TextImputer(text_col, '')),
#                ('selector', FeatureSelector(column=text_col)),
#                ('tfidf', TfidfVectorizer())
#            ])
#    final_transformers.append((text_col, text_transformer))

In [15]:
#Объединим в пайплайн

from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

In [16]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(multi_class='ovr', n_jobs=-1)),   
])

pipeline.fit(X_train, y_train)

Wall time: 2.67 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('Cabin',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Cabin')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Cabin'))])),
                                                ('Embarked',
                                                 Pipeline(steps=[('selector',
                              

In [17]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

## Step 2 - PREDICT
### Проверка работоспособности и качества пайплайна

In [18]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")
X_test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,710,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
1,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
2,841,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S


In [19]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [20]:
y_test['Survived'].unique()

array([1, 0], dtype=int64)

In [21]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [22]:
preds[:10]

array([0.11294452, 0.20338122, 0.12611295, 0.85733642, 0.74829539,
       0.91272715, 0.61800209, 0.0874193 , 0.731411  , 0.90059009])

In [23]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.33721850606237047, F-Score=0.794, Precision=0.745, Recall=0.850


## Step 3 - FLASK

In [24]:
!pip install flask-ngrok

In [25]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [26]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc
!./ngrok http 80

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
tar: Error opening archive: Failed to open '/content/ngrok-stable-linux-amd64.tgz'
"." ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"." ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [27]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://87b7-178-141-108-237.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
